In [8]:
# Mount Google Drive
import os
from google.colab import drive
drive.mount('/content/drive')

# Install dependencies
!pip install ultralytics

# Set paths
base_path = '/content/drive/MyDrive/HackAura'
extract_path = '/content/HackAura_dataset'
os.makedirs(extract_path, exist_ok=True)

# Extract all zip files
import zipfile
zip_files = [f for f in os.listdir(base_path) if f.endswith('.zip')]
if not zip_files:
    raise FileNotFoundError(f"No zip files found in {base_path}. Please upload zip files to Google Drive.")
for zip_file in zip_files:
    zip_path = os.path.join(base_path, zip_file)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Extracted {zip_file} to {extract_path}")

# Set working directory
os.chdir(extract_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extracted Hackathon2_scripts.zip to /content/HackAura_dataset
Extracted hackathon2_test3.zip to /content/HackAura_dataset
Extracted hackathon2_train_3.zip to /content/HackAura_dataset


In [9]:
# Verify yolo_params.yaml exists
config_path = '/content/HackAura_dataset/Hackathon2_scripts/yolo_params.yaml'
if not os.path.exists(config_path):
    raise FileNotFoundError(f"Config file not found at {config_path}. Please check the path and ensure the file exists in Google Drive.")

In [10]:
# Import and train model
from ultralytics import YOLO
import argparse

# Define parameters
EPOCHS = 15  # Balanced for ~1-2 hrs on GPU
MOSAIC = 0.4  # Moderate augmentation
OPTIMIZER = 'AdamW'  # Stable optimizer
MOMENTUM = 0.9  # Standard
LR0 = 0.001  # Initial learning rate
LRF = 0.01  # Final learning rate
SINGLE_CLS = False  # Multi-class

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--epochs', type=int, default=EPOCHS, help='Number of epochs')
    parser.add_argument('--mosaic', type=float, default=MOSAIC, help='Mosaic augmentation')
    parser.add_argument('--optimizer', type=str, default=OPTIMIZER, help='Optimizer')
    parser.add_argument('--momentum', type=float, default=MOMENTUM, help='Momentum')
    parser.add_argument('--lr0', type=float, default=LR0, help='Initial learning rate')
    parser.add_argument('--lrf', type=float, default=LRF, help='Final learning rate')
    parser.add_argument('--single_cls', type=bool, default=SINGLE_CLS, help='Single class training')
    args = parser.parse_args([])  # Empty list to avoid Colab argument parsing issues

    # Load pre-trained model (upload yolov8s.pt if not present)
    model_path = os.path.join(extract_path, "yolov8s.pt")
    if not os.path.exists(model_path):
        model_path = '/content/drive/MyDrive/HackAura/Hackathon2_scripts/yolov8s.pt'  # Fallback to scripts folder
    model = YOLO(model_path)

    # Train the model
    results = model.train(
        data=config_path,  # Use verified config path from Drive
        epochs=args.epochs,
        device='0',  # Use Colab GPU
        single_cls=args.single_cls,
        mosaic=args.mosaic,
        optimizer=args.optimizer,
        lr0=args.lr0,
        lrf=args.lrf,
        momentum=args.momentum,
        imgsz=640,
        batch=16,
        patience=5  # Early stopping
    )

# Evaluate the model
results = model.val()
print(f"mAP@0.5: {results.box.map}")  # Quick performance check

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/HackAura_dataset/Hackathon2_scripts/yolo_params.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640